## Experiment-04
### Amirreza Fosoul and Bithiah Yuan

In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql.functions import *
import string
import csv
import re
import time
spark = SparkSession.builder.appName('ex3').getOrCreate()

In [2]:
from pyspark.sql.functions import *
from pyspark.sql.functions import split, udf, desc, concat, col, lit
import pyspark.sql.functions as f
from pyspark.sql.types import ArrayType, FloatType, StringType, IntegerType, DoubleType, StructType, StructField
from pyspark.ml.feature import RegexTokenizer
from pyspark.ml.feature import StopWordsRemover
from pyspark.sql.window import Window
from pyspark.ml.linalg import SparseVector, VectorUDT, DenseVector
import scipy.sparse
from pyspark.ml.linalg import Vectors, _convert_to_vector, VectorUDT
import numpy as np
from pyspark.sql import SQLContext
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
import math
import re

sc = spark.sparkContext
sqlContext = SQLContext(sc)

#### We reduced our dataset for both the user_libraries and papers due to out of memory issues

In [22]:
# Read user ratings into Dataframe
#user_df = spark.read.option("delimiter", ";").csv('./users_libraries.txt')
user_df = spark.read.option("delimiter", ";").csv('./example2.txt')
user_df = user_df.select(col("_c0").alias("userID"), col("_c1").alias("paperID"))
user_df_pre = user_df
user_df_pre = user_df_pre.withColumn("paperID", split(col("paperID"), ",").cast(ArrayType(IntegerType())).alias("paperID"))
user_df_pre.show()

user_df = user_df.select("userID", f.split("paperID", ",").alias("papers"), f.explode(f.split("paperID", ",")).alias("paperID"))
user_df = user_df.drop("papers")

# Get a dataframe of the distinct papers
d_paper = user_df.select("paperID").distinct()

+--------------------+--------------------+
|              userID|             paperID|
+--------------------+--------------------+
|28d3f81251d94b097...|[3929762, 503574,...|
|d0c9aaa788153daea...|[2080631, 6343346...|
|f05bcffe7951de9e5...|[1158654, 478707,...|
|ca4f1ba4094011d9a...|            [278019]|
|d1d41a15201915503...|[6610569, 6493797...|
|f2f77383828ea6d39...|[943458, 238121, ...|
|9c883d02115400f7b...|[3509971, 3509965...|
|b656009a6efdc8b1a...|[771870, 181369, ...|
|cf9c7f356092c34be...|             [90558]|
|0f5cbb39410a9278f...|           [9344598]|
|d85f7d83f27b3f533...|[7610843, 3633347...|
|586c867a0688250ac...|[464760, 466011, ...|
|10fdfaf945d5c27ad...|           [2010550]|
|589b870a611c25fa9...|[1283233, 1305474...|
|90f1a3e6fcdbf9bc5...|[115945, 11733005...|
|7e070a9da96672e05...|           [1071959]|
|3b715ebaf1f8f81a1...|[4119394, 3378798...|
|488fb15e8c77f8054...|[1523301, 5281566...|
|3fdf355e59949c79d...|[7077220, 1289842...|
|c6b59086a0bbac141...|[2230995, 

In [23]:
# Read in the stopwords as a list
with open('./stopwords_en.txt') as file:
    stopwordList = file.read().splitlines()

# Read in records of paper information
#w_df = spark.read.csv('./papers.csv')
w_df = spark.read.csv('./example_paper.csv')
w_df = w_df.select("_c0", "_c13", "_c14")
w_df = w_df.select(col("_c0").alias("paperID"), col("_c13").alias("title"), col("_c14").alias("abstract"))
w_df = w_df.na.fill({'title': '', 'abstract': ''}) # to replace null values with empty string
# Get text from title and abstract
w_df = w_df.select(col("paperID"), concat(col("title"), lit(" "), col("abstract")).alias("words"))
#w_df.show()

# Transform the distinct paperIDs dataframe to a list
paper_list = list(d_paper.select('paperID').toPandas()['paperID'])
# Map each distinct paper into int
paper_list = list(map(int, paper_list))

In [24]:
# Function to call in udf
def unrated(papers):
    # Transform the list of distinct papers and the list of rated papers of each user to a set
    # Substract the two sets to get the list of unrated papers for each user
    # Transform back to list
    unrated = list(set(paper_list) - set(papers))
    
    return unrated


# udf to get a list of unrated papers with the length of rated papers for each user
get_unrated = udf(lambda x: unrated(x), ArrayType(IntegerType()))

# Add a new column of unrated papers for each user
unrated_df = user_df_pre.withColumn("unrated", get_unrated(user_df_pre.paperID))

unrated_df = unrated_df.drop("paperID")

unrated_df = unrated_df.withColumn("paperID", explode(unrated_df.unrated))

unrated_df = unrated_df.drop("unrated")

unrated_df.show()

+--------------------+-------+
|              userID|paperID|
+--------------------+-------+
|28d3f81251d94b097...|1277953|
|28d3f81251d94b097...|3153930|
|28d3f81251d94b097...|3153931|
|28d3f81251d94b097...|1388555|
|28d3f81251d94b097...|4186128|
|28d3f81251d94b097...|1814546|
|28d3f81251d94b097...|7499794|
|28d3f81251d94b097...| 921623|
|28d3f81251d94b097...| 970776|
|28d3f81251d94b097...| 311321|
|28d3f81251d94b097...|7499801|
|28d3f81251d94b097...| 118812|
|28d3f81251d94b097...| 290846|
|28d3f81251d94b097...|4067359|
|28d3f81251d94b097...| 688160|
|28d3f81251d94b097...|1118240|
|28d3f81251d94b097...|  81954|
|28d3f81251d94b097...|4460578|
|28d3f81251d94b097...| 340004|
|28d3f81251d94b097...|2363430|
+--------------------+-------+
only showing top 20 rows



## Exercise 3. 1 Vector representation for the papers

In [16]:
# Extracting words from the papers and keeping "-" and "_"
tokenizer = RegexTokenizer(inputCol="words", outputCol="tokens", pattern="[a-zA-Z-_]+", gaps=False) 
# Built-in tokenizer
tokenized = tokenizer.transform(w_df)
tokenized = tokenized.select("paperID", "tokens")

# udf to remove "-" and "_" from the tokens
remove_hyphen_udf = udf(lambda x: [re.sub('[-|_]', '', word) for word in x], ArrayType(StringType()))
# Apply udf to the tokens
df = tokenized.withColumn('tokens', remove_hyphen_udf(col('tokens')))

# udf to remove words less than 3 letters
remove_short_words = udf(lambda x: [item for item in x if len(item) >= 3], ArrayType(StringType()))
# Apply udf to the tokens
df = df.withColumn('tokens', remove_short_words(col('tokens')))

# Built-in function to remove stopwords from our custom list
remover = StopWordsRemover(inputCol="tokens", outputCol="filtered" , stopWords=stopwordList)
df = remover.transform(df)
df = df.select("paperID", "filtered")

# Apply stemming with NLTK
# Built-in class from NLTK
ps = PorterStemmer()
# udf to apply stemming
stemming = udf(lambda x: [ps.stem(item) for item in x], ArrayType(StringType()))
# apply udf to tokens
df = df.withColumn('stemmed', stemming(col('filtered')))
df = df.select("paperID", "stemmed")

# Create a new df to store the paperID and stemmed tokens
paper_terms = df

# Explode/Split the tokens in the list for each paperID and get the distinct tokens
df = df.select("paperID", f.explode("stemmed").alias("tokens")).distinct().orderBy("paperID")

# Assign count of 1 to each token w.r.t. the paperID since the tokens are distinct
df = df.groupBy("paperID","tokens").count()

# Get the number of distinct papers
num_papers = w_df.select("paperID").distinct().count()

# Get the value of ten percent of the number of papers
ten_percent = math.ceil(num_papers*.1)

# Create a new df with the tokens and count (without paperID)
df2 = df.select("tokens", "count")
# Count the number of papers containing the tokens
df2 = df2.groupBy("tokens").agg(f.collect_list("tokens").alias("duplicated_values"), f.sum("count").alias("count"))
# Filter out tokens that appeared in more than 10 percent of the papers
df2 = df2.drop("duplicated_values").orderBy((col("count")).desc()).filter(col("count") < ten_percent)
# Filter out tokens that appeared in less than 20 papers
# Limit the df to 1000 tokens
df2 = df2.filter(col("count") >= 20).limit(1000)

# Create a new df with terms and count
important_words = df2.select(col("tokens").alias("terms"), col("count"))
# Output the set of important words, T
#important_words.show(10)

In [6]:
# Create a new df where each term is replaced by a unique index that takes a value from the range between 0 and |T| − 1
df = important_words.withColumn("row_num", row_number().over(Window.orderBy("count"))-1)

# Create a df to store the indices and the corresponding terms
terms_index_hash = df.select(col("row_num").alias("index"), "terms")

In [8]:
num_terms = terms_index_hash.select("terms").distinct().count()

print(num_terms)

64


In [18]:
p_terms = paper_terms.select("paperID", f.explode("stemmed").alias("terms"))

# Join p_terms with the terms_index_hash to replace the terms with the indices
joined_df = terms_index_hash.join(p_terms, ["terms"])
joined_df = joined_df.drop("index")

# Create a new df to compute the term frequency vectors
tf_df = joined_df

tf_df = tf_df.groupby("paperID").agg(f.concat_ws(", ", f.collect_list(tf_df.terms)).alias("terms"))
tf_df = tf_df.withColumn("terms_", split(col("terms"), ",\s*").cast(ArrayType(StringType())).alias("terms"))
tf_df = tf_df.drop("terms")
tf_df.show()


+-------+--------------------+
|paperID|              terms_|
+-------+--------------------+
| 159967|[scale, predict, ...|
|2212959|[map, cluster, cl...|
| 333353|[pattern, current...|
| 438129|  [collect, resourc]|
| 166220|[challeng, curren...|
|2883810|     [offer, featur]|
|1288940|[web, web, web, w...|
|5251453|[pattern, effect,...|
|7515828|[dna, offer, featur]|
|2739852|[web, emerg, scal...|
|5961524|[offer, addit, ev...|
|    272|[current, dynam, ...|
|6573750|       [cell, genet]|
|  77265|   [pattern, evolut]|
| 820297|[simpl, perform, ...|
|1042553|[power, power, po...|
|2883820|[technolog, curre...|
|5434882|[current, emerg, ...|
|1332540|             [simpl]|
|1777140|[map, challeng, p...|
+-------+--------------------+
only showing top 20 rows



In [20]:
from pyspark.ml.feature import CountVectorizer, CountVectorizerModel

# Get the number of distinct terms
num_terms = terms_index_hash.select("terms").distinct().count()

print(num_terms)

cv = CountVectorizer(inputCol="terms_", outputCol="vectors")
model = cv.fit(tf_df)
vector_df = model.transform(tf_df)
vector_df = vector_df.select("paperID", col("vectors").alias("term_frequency_sparse"))

vector_df.show(truncate=False)

64
+-------+-----------------------------------------------------------------------------------------------------------------------------------+
|paperID|term_frequency_sparse                                                                                                              |
+-------+-----------------------------------------------------------------------------------------------------------------------------------+
|159967 |(64,[0,5,16,21,24,35,38,40,44,52,58],[12.0,1.0,3.0,4.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])                                               |
|2212959|(64,[1,16,58],[1.0,3.0,1.0])                                                                                                       |
|333353 |(64,[3,4,13,29,36,37,39,41,43,46,61],[1.0,1.0,5.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])                                                |
|438129 |(64,[22,48],[1.0,1.0])                                                                                                             |
|16

In [21]:
### TF-IDF with built-in function ###

from pyspark.ml.feature import HashingTF, IDF, Tokenizer

idf = IDF(inputCol="term_frequency_sparse", outputCol="features")
idfModel = idf.fit(vector_df)
rescaledData = idfModel.transform(vector_df)
tf_idf_built_in = rescaledData.select("paperID", "features")

tf_idf_built_in.show()

+-------+--------------------+
|paperID|            features|
+-------+--------------------+
| 159967|(64,[0,5,16,21,24...|
|2212959|(64,[1,16,58],[2....|
| 333353|(64,[3,4,13,29,36...|
| 438129|(64,[22,48],[2.45...|
| 166220|(64,[8,12,18,24,2...|
|2883810|(64,[4,46],[2.219...|
|1288940|(64,[2,10,33,38,4...|
|5251453|(64,[0,3,10,11,17...|
|7515828|(64,[4,6,46],[2.2...|
|2739852|(64,[0,2,9,15,16,...|
|5961524|(64,[13,46,56,57,...|
|    272|(64,[7,10,17,20,2...|
|6573750|(64,[19,20],[2.49...|
|  77265|(64,[3,13],[2.255...|
| 820297|(64,[26,36,55],[2...|
|1042553|(64,[5,17,32,40,4...|
|2883820|(64,[14,29,57],[2...|
|5434882|(64,[13,15,18,29,...|
|1332540|(64,[55],[2.40919...|
|1777140|(64,[0,1,5,21,31]...|
+-------+--------------------+
only showing top 20 rows



In [25]:
# create the user profile using the tf_idf dataframe and the users' library dataframe
user_profile = tf_idf_built_in.join(user_df, ["paperID"]).orderBy("userID").select('userId', 'features')

# convert the dataframe to RDD to sum up the tf_idf vector of each user and then convert back to dataframe
user_profile = user_profile.rdd.mapValues(lambda v: v.toArray()).reduceByKey(lambda x, y: x + y).mapValues(lambda x: DenseVector(x))\
.toDF(["userId", "features"])

user_profile = user_profile.select("userId", col("features").alias("user_profile"))


In [26]:
def to_sparse(x):        
    # store the indices of non-zero elements
    nonzero_indices = np.nonzero(x)[0].tolist()
    # store the value of non-zero elements
    nonzero_counts = [num for num in x if num]
    # combine them to make a sparse vector
    sparse = SparseVector(num_terms, nonzero_indices, nonzero_counts)
    return sparse

to_sparse_udf = udf(lambda x: to_sparse(x), VectorUDT())

user_profile = user_profile.withColumn("user_profile", to_sparse_udf(col("user_profile")))

user_profile.show()

+--------------------+--------------------+
|              userId|        user_profile|
+--------------------+--------------------+
|d503571e44a0373eb...|(64,[6,11,13,19,3...|
|f1e1cd4ff25018273...|(64,[8,9,12,18,37...|
|bbcd9dae3160ddcb9...|(64,[5,9,18,36,42...|
|a0bbf6bb9b1c818f3...|(64,[5,18,42],[8....|
|1eac022a97d683eac...|(64,[0,4,5,6,10,1...|
|cf9c7f356092c34be...|(64,[2,4,19,23,30...|
|3b715ebaf1f8f81a1...|(64,[2,8,9,10,12,...|
|b36c3189bb1457cd0...|(64,[46],[2.45175...|
|f3c28e50db4ce8ad8...|(64,[6,11,13,19,3...|
|b656009a6efdc8b1a...|(64,[2,3,4,6,7,8,...|
|d85f7d83f27b3f533...|(64,[17,18,20,28,...|
|f05bcffe7951de9e5...|(64,[0,1,2,4,5,6,...|
|4c8912d1b04471cf5...|(64,[0,3,4,6,10,2...|
+--------------------+--------------------+



In [27]:
df = unrated_df.join(tf_idf_built_in, ["paperID"]).join(user_profile, ["userID"])

df = df.select("userID", "paperID", "user_profile", col("features").alias("paper_profile"))

df.show()

+--------------------+-------+--------------------+--------------------+
|              userID|paperID|        user_profile|       paper_profile|
+--------------------+-------+--------------------+--------------------+
|f1e1cd4ff25018273...| 166220|(64,[8,9,12,18,37...|(64,[8,12,18,24,2...|
|f1e1cd4ff25018273...| 820297|(64,[8,9,12,18,37...|(64,[26,36,55],[2...|
|f1e1cd4ff25018273...|1042553|(64,[8,9,12,18,37...|(64,[5,17,32,40,4...|
|f1e1cd4ff25018273...|3010240|(64,[8,9,12,18,37...|(64,[18,50],[2.45...|
|f1e1cd4ff25018273...| 523772|(64,[8,9,12,18,37...|(64,[55],[2.40919...|
|f1e1cd4ff25018273...|1279898|(64,[8,9,12,18,37...|(64,[3,43,46,47],...|
|f1e1cd4ff25018273...| 920055|(64,[8,9,12,18,37...|(64,[4,12,46],[2....|
|f1e1cd4ff25018273...| 965334|(64,[8,9,12,18,37...|(64,[4,13,34,47],...|
|f1e1cd4ff25018273...|     99|(64,[8,9,12,18,37...|(64,[7,8,14,16,17...|
|f1e1cd4ff25018273...|7355647|(64,[8,9,12,18,37...|(64,[0,7,9,15,44,...|
|f1e1cd4ff25018273...|   4280|(64,[8,9,12,18,37...|

In [28]:
df = unrated_df.join(tf_idf_built_in, ["paperID"]).join(user_profile, ["userID"])

df = df.select("userID", "paperID", "user_profile", col("features").alias("paper_profile"))

# def to_dense(x):
#     return DenseVector(x.toArray())

# to_dense_udf = udf(lambda x: to_dense(x), VectorUDT())

# df = df.withColumn('paper_profile_dense', to_dense_udf(df.paper_profile))
# df = df.drop("paper_profile")

df_selected = df.where(df.userID=="1eac022a97d683eace8815545ce3153f")

df_selected.show()

+--------------------+--------+--------------------+--------------------+
|              userID| paperID|        user_profile|       paper_profile|
+--------------------+--------+--------------------+--------------------+
|1eac022a97d683eac...|  166220|(64,[0,4,5,6,10,1...|(64,[8,12,18,24,2...|
|1eac022a97d683eac...|  820297|(64,[0,4,5,6,10,1...|(64,[26,36,55],[2...|
|1eac022a97d683eac...| 1279898|(64,[0,4,5,6,10,1...|(64,[3,43,46,47],...|
|1eac022a97d683eac...|  920055|(64,[0,4,5,6,10,1...|(64,[4,12,46],[2....|
|1eac022a97d683eac...|  965334|(64,[0,4,5,6,10,1...|(64,[4,13,34,47],...|
|1eac022a97d683eac...|      99|(64,[0,4,5,6,10,1...|(64,[7,8,14,16,17...|
|1eac022a97d683eac...| 7355647|(64,[0,4,5,6,10,1...|(64,[0,7,9,15,44,...|
|1eac022a97d683eac...|    4280|(64,[0,4,5,6,10,1...|(64,[46],[2.45175...|
|1eac022a97d683eac...| 4302361|(64,[0,4,5,6,10,1...|(64,[18],[2.45175...|
|1eac022a97d683eac...|   72879|(64,[0,4,5,6,10,1...|(64,[5,29,35,46],...|
|1eac022a97d683eac...|   90558|(64,[0,

In [29]:
import numpy as np 

def cos_sim(u, p):
    result = (np.dot(u, p))/(np.linalg.norm(u) * np.linalg.norm(p))
    result = result.item()
    return result

compute_sim = udf(cos_sim, FloatType())

def cbrs_tf_idf(u, k):
    sim_df = u.withColumn('Similarity', compute_sim(u.user_profile, u.paper_profile))
    window = Window.partitionBy(col("userID")).orderBy((col("Similarity")).desc())
    sim_df = sim_df.select(col('*'), row_number().over(window).alias('row_number')).where(col('row_number') <= k)
    get_r = sim_df.select("userID", "paperID", col("row_number").alias("rank"))
    cbrs = get_r.select("userID", "paperID")
    cbrs = cbrs.groupby("userID").agg(f.concat_ws(", ", f.collect_list(cbrs.paperID)).alias("top_papers"))
    
    print("The top " + str(k) + " papers for " + str(cbrs.head()[0]) + " are " + str(cbrs.head()[1]))

cbrs_tf_idf(df_selected, 5)

The top 5 papers for 1eac022a97d683eace8815545ce3153f are 100088, 211804, 557229, 478707, 4302361


In [30]:
### LDA ###
from pyspark.ml.clustering import LDA

# just rename the column to use it with the built-in methods
termFrequencyVector = vector_df.select('paperId', col('term_frequency_sparse').alias('features'))
# Trains a LDA model
# set k=40 to have 40 different topics
lda = LDA(k=40)
model = lda.fit(termFrequencyVector)

# each topic is described by 5 terms
topics = model.describeTopics(5)

# print("The topics described by their top-weighted terms:")
#topics.show(truncate=False)

# Shows the result
# it shows the probabilty of each topic for each paper
transformed = model.transform(termFrequencyVector)
transformed.show()

+-------+--------------------+--------------------+
|paperId|            features|   topicDistribution|
+-------+--------------------+--------------------+
| 159967|(64,[0,5,16,21,24...|[8.62882233031433...|
|2212959|(64,[1,16,58],[1....|[0.00403466100484...|
| 333353|(64,[3,4,13,29,36...|[0.00151064718998...|
| 438129|(64,[22,48],[1.0,...|[0.00808946256218...|
| 166220|(64,[8,12,18,24,2...|[0.00219823645336...|
|2883810|(64,[4,46],[1.0,1...|[0.00808946256218...|
|1288940|(64,[2,10,33,38,4...|[0.00219823645336...|
|5251453|(64,[0,3,10,11,17...|[0.00142170763106...|
|7515828|(64,[4,6,46],[1.0...|[0.00605953480064...|
|2739852|(64,[0,2,9,15,16,...|[0.00100678528591...|
|5961524|(64,[13,46,56,57,...|[0.00241838815250...|
|    272|(64,[7,10,17,20,2...|[0.00161145714901...|
|6573750|(64,[19,20],[1.0,...|[0.00808946256218...|
|  77265|(64,[3,13],[1.0,1...|[0.00808946256218...|
| 820297|(64,[26,36,55],[1...|[0.00605953480064...|
|1042553|(64,[5,17,32,40,4...|[0.00142170763106...|
|2883820|(64

In [31]:
# creating a user profile based on the LDA results

lda_user_profile = transformed.join(user_df, ["paperID"]).orderBy("userID").select('userId', col('topicDistribution').alias('features'))

lda_user_profile = lda_user_profile.rdd.mapValues(lambda v: v.toArray()).reduceByKey(lambda x, y: x + y).mapValues(lambda x: DenseVector(x))\
.toDF(["userId", "features"])
lda_user_profile = lda_user_profile.select("userId", col("features").alias("lda_user_profile"))

lda_user_profile = lda_user_profile.withColumn("user_profile", to_sparse_udf(col("lda_user_profile")))

lda_user_profile.show()

+--------------------+--------------------+--------------------+
|              userId|    lda_user_profile|        user_profile|
+--------------------+--------------------+--------------------+
|d503571e44a0373eb...|[0.00172668387563...|(64,[0,1,2,3,4,5,...|
|f1e1cd4ff25018273...|[0.02428867942528...|(64,[0,1,2,3,4,5,...|
|bbcd9dae3160ddcb9...|[0.01922333031259...|(64,[0,1,2,3,4,5,...|
|a0bbf6bb9b1c818f3...|[0.00302411344949...|(64,[0,1,2,3,4,5,...|
|1eac022a97d683eac...|[0.04247925079504...|(64,[0,1,2,3,4,5,...|
|cf9c7f356092c34be...|[0.00172668387563...|(64,[0,1,2,3,4,5,...|
|3b715ebaf1f8f81a1...|[0.01709020374542...|(64,[0,1,2,3,4,5,...|
|b36c3189bb1457cd0...|[0.01216455585825...|(64,[0,1,2,3,4,5,...|
|f3c28e50db4ce8ad8...|[0.00172668387563...|(64,[0,1,2,3,4,5,...|
|b656009a6efdc8b1a...|[0.06713783507144...|(64,[0,1,2,3,4,5,...|
|d85f7d83f27b3f533...|[0.00830105652950...|(64,[0,1,2,3,4,5,...|
|f05bcffe7951de9e5...|[0.03103687891449...|(64,[0,1,2,3,4,5,...|
|4c8912d1b04471cf5...|[0.

In [32]:
df_lda = unrated_df.join(tf_idf_built_in, ["paperID"]).join(lda_user_profile, ["userID"])

df_lda = df_lda.select("userID", "paperID", "user_profile", col("features").alias("paper_profile"))

#df_lda.show()

In [33]:
df_selected_lda = df_lda.where(df_lda.userID=="1eac022a97d683eace8815545ce3153f")

df_selected_lda.show()

+--------------------+--------+--------------------+--------------------+
|              userID| paperID|        user_profile|       paper_profile|
+--------------------+--------+--------------------+--------------------+
|1eac022a97d683eac...|  166220|(64,[0,1,2,3,4,5,...|(64,[8,12,18,24,2...|
|1eac022a97d683eac...|  820297|(64,[0,1,2,3,4,5,...|(64,[26,36,55],[2...|
|1eac022a97d683eac...| 1279898|(64,[0,1,2,3,4,5,...|(64,[3,43,46,47],...|
|1eac022a97d683eac...|  920055|(64,[0,1,2,3,4,5,...|(64,[4,12,46],[2....|
|1eac022a97d683eac...|  965334|(64,[0,1,2,3,4,5,...|(64,[4,13,34,47],...|
|1eac022a97d683eac...|      99|(64,[0,1,2,3,4,5,...|(64,[7,8,14,16,17...|
|1eac022a97d683eac...| 7355647|(64,[0,1,2,3,4,5,...|(64,[0,7,9,15,44,...|
|1eac022a97d683eac...|    4280|(64,[0,1,2,3,4,5,...|(64,[46],[2.45175...|
|1eac022a97d683eac...| 4302361|(64,[0,1,2,3,4,5,...|(64,[18],[2.45175...|
|1eac022a97d683eac...|   72879|(64,[0,1,2,3,4,5,...|(64,[5,29,35,46],...|
|1eac022a97d683eac...|   90558|(64,[0,

In [34]:
cbrs_tf_idf(df_selected_lda, 5)

The top 5 papers for 1eac022a97d683eace8815545ce3153f are 81501, 129, 90558, 11191048, 80546


## Exercise 4. 4 Sampling and data preparation

In [ ]:
sampled_user_profile = user_profile.sample(False, 1).limit(10)
sampled_user_profile.show()